In [1]:
from judo_footage_analysis.utils import get_spark

root = "/cs-share/pradalier/tmp/judo"

spark = get_spark()
spark

24/04/05 13:32:59 WARN Utils: Your hostname, gtlpc108.georgiatech-metz.fr resolves to a loopback address: 127.0.1.1; using 192.93.8.108 instead (on interface enp0s31f6)
24/04/05 13:32:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/05 13:33:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/05 13:33:00 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
from pyspark.sql import functions as F

df = spark.read.parquet(f"{root}/data/evaluation_embeddings/v2").cache()
df.printSchema()
df.show()

root
 |-- file: string (nullable = true)
 |-- time: long (nullable = true)
 |-- is_active: long (nullable = true)
 |-- is_match: long (nullable = true)
 |-- is_standing: long (nullable = true)
 |-- emb_entity_detection_v2: vector (nullable = true)
 |-- emb_entity_detection_v2_dct: vector (nullable = true)
 |-- emb_entity_detection_v2_dct_d8: vector (nullable = true)
 |-- emb_vanilla_yolov8n: vector (nullable = true)
 |-- emb_vanilla_yolov8n_dct: vector (nullable = true)
 |-- emb_vanilla_yolov8n_dct_d8: vector (nullable = true)



+--------------------+----+---------+--------+-----------+-----------------------+---------------------------+------------------------------+--------------------+-----------------------+--------------------------+
|                file|time|is_active|is_match|is_standing|emb_entity_detection_v2|emb_entity_detection_v2_dct|emb_entity_detection_v2_dct_d8| emb_vanilla_yolov8n|emb_vanilla_yolov8n_dct|emb_vanilla_yolov8n_dct_d8|
+--------------------+----+---------+--------+-----------+-----------------------+---------------------------+------------------------------+--------------------+-----------------------+--------------------------+
|data/clips/mat_01...|   4|        0|       0|          0|   [-12.463060379028...|       [-1822.2849139964...|          [-1822.2849139964...|[-12.463060379028...|   [-1822.2849139964...|      [-1822.2849139964...|
|data/clips/mat_01...|   7|        0|       0|          0|   [-12.700770378112...|       [-1819.9931834956...|          [-1819.9931834956...|[-1

In [12]:
# let's create time lagged features for each embedding
from pyspark.sql import Window, functions as F

window = Window.partitionBy("file").orderBy("time")
lagged = (
    df.withColumn("lag0", F.lag("emb_entity_detection_v2_dct_d8", 0).over(window))
    .withColumn("lag1", F.lag("emb_entity_detection_v2_dct_d8", 1).over(window))
    .withColumn("lag2", F.lag("emb_entity_detection_v2_dct_d8", 2).over(window))
    .withColumn("lag3", F.lag("emb_entity_detection_v2_dct_d8", 3).over(window))
).select("file", "time", "lag0", "lag1", "lag2", "lag3")
lagged.show()

+--------------------+----+--------------------+--------------------+--------------------+--------------------+
|                file|time|                lag0|                lag1|                lag2|                lag3|
+--------------------+----+--------------------+--------------------+--------------------+--------------------+
|data/clips/mat_01...|   0|[-1821.6943279371...|                NULL|                NULL|                NULL|
|data/clips/mat_01...|   1|[-1822.6560198885...|[-1821.6943279371...|                NULL|                NULL|
|data/clips/mat_01...|   2|[-1810.5051809256...|[-1822.6560198885...|[-1821.6943279371...|                NULL|
|data/clips/mat_01...|   3|[-1814.6923456057...|[-1810.5051809256...|[-1822.6560198885...|[-1821.6943279371...|
|data/clips/mat_01...|   4|[-1822.2849139964...|[-1814.6923456057...|[-1810.5051809256...|[-1822.6560198885...|
|data/clips/mat_01...|   5|[-1810.0977286660...|[-1822.2849139964...|[-1814.6923456057...|[-1810.5051809

In [13]:
# only keep rows where there are no nulls
lagged.count(), lagged.filter(F.col("lag3").isNotNull()).count()

(570, 513)

In [14]:
spark.stop()